In [1]:
# 0. Dependencies
import itertools
import time
import binascii
from random import randrange
import operator

In [2]:
# 1. Helper functions
def getReads(f):
    '''Parses a reads file'''
    reads = []
    while True:
        first_line =  f.readline()
        if len(first_line) == 0:
            break  # end of file
        name = first_line[1:].rstrip()
        seq = f.readline().rstrip()
        # f.readline()  # ignore line starting with +
        # qual = f.readline().rstrip()
        reads.append(seq)
    return reads

def makeIndex(read, index, mini, maxi):
    '''Creates an index table'''
    index.append({})
    for i in range(len(read)):
        for j in range(mini,maxi):
            word = read[i:i+j]
            if len(word) == j:
                if word not in index[-1]:
                    index[-1][word] = [i]
                else:
                    index[-1][word].append(i)
    return index

In [3]:
# 2. SW Filtering
def shortWordFilter(reads, index, mini, maxi):
    '''Short Word Filtering'''
    cluster = []
    cluster.append([reads[0]])
    reads.pop(0)

    while(reads):
        read = reads.pop(0)
        L = len(read)
        similar = False 
        for x in range(len(index)):
            counts = [0] * 4
            for i in range(len(read)):
                for j in range(mini,maxi+1):
                    word = read[i:i+j]
                    if len(word) == j:
                        if word in index[x]:
                            counts[j-mini] += 1
            if counts[0] > L-mini+1-(.02)*mini*L:
                cluster[x].append(read)
                similar = True
                break;
        if not similar:
            cluster.append([read])
            index = makeIndex(read, index, mini, maxi+1)
            
    return cluster

In [4]:
# 3.1 Rosetta Code LCS. O(mn) space, O(mn) time. Does not work. DO NOT USE
def llcs1(a,b):
    return float(len(lcs1(a,b)))/float((min([len(a), len(b)])))

def lcs1(a, b):
    lengths = [[0 for j in range(len(b)+1)] for i in range(len(a)+1)]
    
    # row 0 and column 0 are initialized to 0 already
    for i, x in enumerate(a):
        for j, y in enumerate(b):
            if x == y:
                lengths[i+1][j+1] = lengths[i][j] + 1
            else:
                lengths[i+1][j+1] = max(lengths[i+1][j], lengths[i][j+1])

    # read the substring out from the matrix
    result = ""

    x, y = len(a), len(b)
    while x != 0 and y != 0:
        if lengths[x][y] == lengths[x-1][y]:
            x -= 1
        elif lengths[x][y] == lengths[x][y-1]:
            y -= 1
        else:
            assert a[x-1] == b[y-1]
            result = a[x-1] + result
            x -= 1
            y -= 1
    return result

print lcs1('human','chimpanzee')
print lcs1('ATCTGAT', 'TGCATA')

hman
TCTA


In [5]:
# 3.2 My LCS. Calculated recursively. O(mn) space, O(mn) time. Slow.
def llcs2(a,b):
    return float(len("".join(lcs2(a,b))))/float((min([len(a), len(b)])))

def lcs2(xs, ys):
    if xs and ys:
        xb, xe = xs[:-1], xs[-1]
        yb, ye = ys[:-1], ys[-1]
        if xe == ye:
            return lcs2(xb, yb) + [xe]
        else:
            return max(lcs2(xs, yb), lcs2(xb, ys), key=len)
    else:
        return []

print "".join(lcs2('human','chimpanzee'))
print "".join(lcs2('ATCTGAT', 'TGCATA'))

hman
TGAT


In [6]:
# 3.3 My LCS. Dynamic Programming. O(mn) space, O(mn) time. Average
from collections import defaultdict, namedtuple
from itertools import product

def llcs3(a,b):
    return float(len("".join(lcs3(a,b))))/float((min([len(a), len(b)])))

def lcs_grid(xs, ys):
    '''Create a grid for longest common subsequence calculations.
    
    Returns a grid where grid[(j, i)] is a pair (n, move) such that
    - n is the length of the LCS of prefixes xs[:i], ys[:j]
    - move is \, ^, <, or e, depending on whether the best move
      to (j, i) was diagonal, downwards, or rightwards, or None.
    
    Example:
       T  A  R  O  T
    A 0< 1\ 1< 1< 1<
    R 0< 1^ 2\ 2< 2<
    T 1\ 1< 2^ 2< 3\
    '''
    Cell = namedtuple('Cell', 'length move')
    grid = defaultdict(lambda: Cell(0, 'e'))
    sqs = product(enumerate(ys), enumerate(xs))
    for (j, y), (i, x) in sqs:
        if x == y:
            cell = Cell(grid[(j-1, i-1)].length + 1, '\\')
        else:
            left = grid[(j, i-1)].length
            over = grid[(j-1, i)].length
            if left < over:
                cell = Cell(over, '^')
            else:
                cell = Cell(left, '<')
        grid[(j, i)] = cell
    return grid

def lcs3(xs, ys):
    grid = lcs_grid(xs, ys)
    i, j = len(xs) - 1, len(ys) - 1
    lcs = list()
    for move in iter(lambda: grid[(j, i)].move, 'e'):
        if move == '\\':
            lcs.append(xs[i])
            i -= 1
            j -= 1
        elif move == '^':
            j -= 1
        elif move == '<':
            i -= 1
    lcs.reverse()
    return lcs

print "".join(lcs3('human','chimpanzee'))
print "".join(lcs3('ATCTGAT', 'TGCATA'))

# Really long reads
#import time
#longRead1 = "GGGGGCATGCCTTAACATGCAGTCGAACGGCAGCACGGGTGCTTGCACCTGGTGGCGAGTGGCGAACGGGTGAGTAATACATCGGAACATGTCCAGTAGTGGGGGATAGCCCGGCGAAAGCCGGATTAATACCGCATACGCTCTATGGAGGAAAGCGGGGGATCGAAAGACCTCGCGCTATAGGGGTGGCCGATGGCGGATTAGCTAGTTGGTGGGGTAAAGGCCTACCAAGGCGACGATCCGTAGCTGGTCTGAGAGGACGACCAGCCACACTGGGACTGAGACACGGCCCAGACTCCTACGGGAGGCAGCAGTGGGGAATTTTGGACAATGGGCGAAAGCCTGATCCAGCAATGCCGCGTGTGTGAAGAAGGCCTTCGGGTTGTAAAGCACTTTTGTCCGGAAAGAAATCCCTGGTCCTAATATGGCCGGGGGATGACGGTACCGGAAGAATAAGCACCGGCTAACTACGTGCCAGCAGCCGCGGTAATACGTAGGGTGCAAGCGTTAATCGGAATTACTGGGCGTAAAGCGTGCGCAGGCGGTGATGCAAGACCGATGTGAAATCCCCGGGCTCAACCTGGGAACTGCATTGGTGACTGCATCGCTAGAGTATGGCAGAGGGGGGTAGAATTCCACGTGTAGCAGTGAAATGCGTAGAGATGTGGAGGAATACCGATGGCGAAGGCAGCCCCCTGGGTCAATACTGACGCTCATGCACGAAAGCGTGGGGAGCAAACAGGATTAGATACCCTGGTAGTCCACGCCCTAAACGATGTCAACTGGTTGTCGGGTCTTCATTGACTTGGTAACGTAGCTAACGCGTGAAGTTGACCGCCTGGGGAGTACGGTCGCAAGATTAAAACTCAAAGGAATTGACGGGGACCCGCACAAGCGGTGGATGATGTGGATTAATTCGATGCAACGCGAAAAACCTTACCTACCCTTGACATGGATGGAATCCCGCTGAGAGGTGGGAGTGCCCGAAAGGGAGCCATCACACAGGTGCTGCATGGCTGTCGTCAGCTCGTGTCGTGAGATGTTGGGTTAAGTCCCGCAACGAGCGCAACCCTTGTCCCTAGTTGCTACGCAAGAGCACTCCAGGGAGACTGCCGGTGACAAACCGGAGGAAGGTGGGGATGACGTCAAGTCCTCATGGCCCTTATGGGTAGGGCTTCACACGTCATACAATGGTCGGAACAGAGGGTTGCCAAGCCGCGAGGTGGAGCCAATCCCAGAAAACCGATCGTAGTCCGGATCGCAGTCTGCAACTCGACTGCGTGAAGCTGGAATCGCTAGTAATCGCGGATCAGCATGCCGCGGTGAATACGTTCCCGGGTCTTGTACACACCGCCCGTCACACCATGGGAGTGGGTTTTGCCAGAAGTGGCTAGTCTAACCGCAAGGAGGACGGTCACCACGTCAGATCAGCGT"
#longRead2 = "TAGACTGCAGTCGAACGCAGTCTTCGGACTGAGTGGCGCACGGGTGAGTAACGCGTAACTGACGTACCCGGAAGTCCAGAATAACTTCCCGAAAGGGAAGCTAATACCGGATGTGCAGCACTTCTTCTGGGAGTGTTGTAAAGATTTATCGCTTTCGGATCGGGTTGCGTTCCATCAGCTAGATGGTGAGGTAAAGGCTCACCATGGCAACGACGGATAGCCGACCTGAGAGGGTGGTCGGCCACAGGGGCACTGAGACACGGGCCCCACTCCTACGGGAGGCAGCAGTTAGGAATCTTCTGCAATGGACGAAAGTCTGACAGAGCGACGCCGCTTGAGGGAAGAAGGTCTTCGGATTGTAAACCTCTGAACTTGGGACGAAAGACCAGCTTGACTGGGGATGACGGTACCAAGGTAATAGCACCGGCTAACTCCGTGCCAGCAGCCGCGGTAATACGGAGGGTGCGAGCGTTACCCGGAATCACTGGGCGTAAAGGGCGTGTAGGCGGTTTTTCAAGTCTGGCTTTAAAGACCGGAGCTCAACTCCGGGAGTGGGCTGGATACTGGGAGACTTGACGGCTGGAGAGGCAACTGGAATTCCTGGTGTAGCGGTGGAATGCGTAGATACCAGGAGGAACACCGATGGCGAAGGCAGGTTGCTGGACAGCGAGTGACGCTGAGGCGCGAAAGTGTGGGGAGCGAACCGGATTAGATACCCGGGTAGTCCACACCCTAAACGATGCACGTTGGATTACCGTGGGATGCCATGGTGGTCGAAGCCAACGCGATAAACGTGCCGCCTGGGAAGTACGGCCGCAAGGTTGAAACTCAAAGGAATTGACGGGGGCCCGCACAAGCGGTGGAGCATGTGGTTTAATTCGAAGCAACGCGAAGAACCTTACCAGGTCTTGACATGCACGGAGCCTTCCTGAAAGGGAAGGGTGCCCTTCGGGGAACCGTGACACAGGTGCTGCATGGCTGTCGTCAGCTCGTGTCGTGAGATGTTGGGTTAAGTCCCGCAACGAGCGCAACCCTTGCTTTGTGTTACTAACGGTTTGGCCGAGGACTCACAAGGGACTGCCTGTGAAAGCAGGAGGAAGGCGGGGATGACGTCTAGTCAGCATGGTCCTTACGACCTGGGCTACACACGTGCTACAATGGTCGGTACAACGCGCAGCAACACCGCGAGGTGAAGCGAATCGCTGAAAGCCGGCCTCAGTTCAGATTGCAGTCTGCAACTCGACTGCATGAAGTTGGAATCGCTAGTAATCGCGGGTCAGCACACCGCGGTGAATACGTTCCCGGGCCTTGTACACACCGCCCGTCACACCATGGGAGTTGATTGCAGCTGAAGCCGCCGGGAGCCGCAAGGCAGGCGTCTAGGCTGTGGTTGATGACTGGGGTGAAGTCGTAACAA"
#start = time.time()
#print "LCS: ", "".join(lcs3(longRead1, longRead2))
#end = time.time()
#print "LCS Similarity: ", llcs3(longRead1, longRead2)
#print "Run Time: ", end-start, " seconds"

hman
TCTA


In [7]:
# 3.4 Langmead's LCS Example.

def llcs4(a,b):
    return float(len(lcs4(a,b)))/float((min([len(a), len(b)])))

def traceLcs(D, x, y):
    ''' Backtrace for LCS; returns LCS as string. '''
    i, j = len(x), len(y) # start in lower-right
    st = []
    while i > 0 and j > 0:
        # get the three contributions
        diag, vert, horz = 0, 0, 0
        if i > 0 and j > 0:
            delt = -1 if x[i-1] == y[j-1] else 1
            diag = D[i-1, j-1] + delt
        if i > 0: vert = D[i-1, j]
        if j > 0: horz = D[i, j-1]
        if diag <= vert and diag <= horz:
            # diagonal is best, thus, this char is part of LCS
            st.append(x[i-1])
            i -= 1; j -= 1 # move up and left
        elif vert <= horz: i -= 1 # vertical is best; move up
        else: j -= 1 # horizontal is best; move left
    # reverse it, then return string-ized LCS
    return (''.join(st))[::-1]

import numpy
def lcs4(x, y):
    ''' Return LCS of x and y.  Uses bottom-up dynamic programming
        approach. '''
    D = numpy.zeros((len(x)+1, len(y)+1), dtype=int)
    for i in xrange(1, len(x)+1):
        for j in xrange(1, len(y)+1):
            delt = -1 if x[i-1] == y[j-1] else 1
            D[i, j] = min(D[i-1, j-1] + delt, D[i-1, j], D[i, j-1])
    return traceLcs(D, x, y), D

print lcs4('human','chimpanzee')[0]
print lcs4('ATCTGAT', 'TGCATA')[0]

# Really long reads
#import time
#longRead1 = "GGGGGCATGCCTTAACATGCAGTCGAACGGCAGCACGGGTGCTTGCACCTGGTGGCGAGTGGCGAACGGGTGAGTAATACATCGGAACATGTCCAGTAGTGGGGGATAGCCCGGCGAAAGCCGGATTAATACCGCATACGCTCTATGGAGGAAAGCGGGGGATCGAAAGACCTCGCGCTATAGGGGTGGCCGATGGCGGATTAGCTAGTTGGTGGGGTAAAGGCCTACCAAGGCGACGATCCGTAGCTGGTCTGAGAGGACGACCAGCCACACTGGGACTGAGACACGGCCCAGACTCCTACGGGAGGCAGCAGTGGGGAATTTTGGACAATGGGCGAAAGCCTGATCCAGCAATGCCGCGTGTGTGAAGAAGGCCTTCGGGTTGTAAAGCACTTTTGTCCGGAAAGAAATCCCTGGTCCTAATATGGCCGGGGGATGACGGTACCGGAAGAATAAGCACCGGCTAACTACGTGCCAGCAGCCGCGGTAATACGTAGGGTGCAAGCGTTAATCGGAATTACTGGGCGTAAAGCGTGCGCAGGCGGTGATGCAAGACCGATGTGAAATCCCCGGGCTCAACCTGGGAACTGCATTGGTGACTGCATCGCTAGAGTATGGCAGAGGGGGGTAGAATTCCACGTGTAGCAGTGAAATGCGTAGAGATGTGGAGGAATACCGATGGCGAAGGCAGCCCCCTGGGTCAATACTGACGCTCATGCACGAAAGCGTGGGGAGCAAACAGGATTAGATACCCTGGTAGTCCACGCCCTAAACGATGTCAACTGGTTGTCGGGTCTTCATTGACTTGGTAACGTAGCTAACGCGTGAAGTTGACCGCCTGGGGAGTACGGTCGCAAGATTAAAACTCAAAGGAATTGACGGGGACCCGCACAAGCGGTGGATGATGTGGATTAATTCGATGCAACGCGAAAAACCTTACCTACCCTTGACATGGATGGAATCCCGCTGAGAGGTGGGAGTGCCCGAAAGGGAGCCATCACACAGGTGCTGCATGGCTGTCGTCAGCTCGTGTCGTGAGATGTTGGGTTAAGTCCCGCAACGAGCGCAACCCTTGTCCCTAGTTGCTACGCAAGAGCACTCCAGGGAGACTGCCGGTGACAAACCGGAGGAAGGTGGGGATGACGTCAAGTCCTCATGGCCCTTATGGGTAGGGCTTCACACGTCATACAATGGTCGGAACAGAGGGTTGCCAAGCCGCGAGGTGGAGCCAATCCCAGAAAACCGATCGTAGTCCGGATCGCAGTCTGCAACTCGACTGCGTGAAGCTGGAATCGCTAGTAATCGCGGATCAGCATGCCGCGGTGAATACGTTCCCGGGTCTTGTACACACCGCCCGTCACACCATGGGAGTGGGTTTTGCCAGAAGTGGCTAGTCTAACCGCAAGGAGGACGGTCACCACGTCAGATCAGCGT"
#longRead2 = "TAGACTGCAGTCGAACGCAGTCTTCGGACTGAGTGGCGCACGGGTGAGTAACGCGTAACTGACGTACCCGGAAGTCCAGAATAACTTCCCGAAAGGGAAGCTAATACCGGATGTGCAGCACTTCTTCTGGGAGTGTTGTAAAGATTTATCGCTTTCGGATCGGGTTGCGTTCCATCAGCTAGATGGTGAGGTAAAGGCTCACCATGGCAACGACGGATAGCCGACCTGAGAGGGTGGTCGGCCACAGGGGCACTGAGACACGGGCCCCACTCCTACGGGAGGCAGCAGTTAGGAATCTTCTGCAATGGACGAAAGTCTGACAGAGCGACGCCGCTTGAGGGAAGAAGGTCTTCGGATTGTAAACCTCTGAACTTGGGACGAAAGACCAGCTTGACTGGGGATGACGGTACCAAGGTAATAGCACCGGCTAACTCCGTGCCAGCAGCCGCGGTAATACGGAGGGTGCGAGCGTTACCCGGAATCACTGGGCGTAAAGGGCGTGTAGGCGGTTTTTCAAGTCTGGCTTTAAAGACCGGAGCTCAACTCCGGGAGTGGGCTGGATACTGGGAGACTTGACGGCTGGAGAGGCAACTGGAATTCCTGGTGTAGCGGTGGAATGCGTAGATACCAGGAGGAACACCGATGGCGAAGGCAGGTTGCTGGACAGCGAGTGACGCTGAGGCGCGAAAGTGTGGGGAGCGAACCGGATTAGATACCCGGGTAGTCCACACCCTAAACGATGCACGTTGGATTACCGTGGGATGCCATGGTGGTCGAAGCCAACGCGATAAACGTGCCGCCTGGGAAGTACGGCCGCAAGGTTGAAACTCAAAGGAATTGACGGGGGCCCGCACAAGCGGTGGAGCATGTGGTTTAATTCGAAGCAACGCGAAGAACCTTACCAGGTCTTGACATGCACGGAGCCTTCCTGAAAGGGAAGGGTGCCCTTCGGGGAACCGTGACACAGGTGCTGCATGGCTGTCGTCAGCTCGTGTCGTGAGATGTTGGGTTAAGTCCCGCAACGAGCGCAACCCTTGCTTTGTGTTACTAACGGTTTGGCCGAGGACTCACAAGGGACTGCCTGTGAAAGCAGGAGGAAGGCGGGGATGACGTCTAGTCAGCATGGTCCTTACGACCTGGGCTACACACGTGCTACAATGGTCGGTACAACGCGCAGCAACACCGCGAGGTGAAGCGAATCGCTGAAAGCCGGCCTCAGTTCAGATTGCAGTCTGCAACTCGACTGCATGAAGTTGGAATCGCTAGTAATCGCGGGTCAGCACACCGCGGTGAATACGTTCCCGGGCCTTGTACACACCGCCCGTCACACCATGGGAGTTGATTGCAGCTGAAGCCGCCGGGAGCCGCAAGGCAGGCGTCTAGGCTGTGGTTGATGACTGGGGTGAAGTCGTAACAA"
#start = time.time()
#print "LCS: ", lcs4(longRead1, longRead2)[0]
#end = time.time()
#print "LCS Similarity: ", llcs4(longRead1, longRead2)
#print "Run Time: ", end-start, " seconds"

hman
TCTA


In [8]:
# 3.5 LCS. Hirschberg's algorithm. O(mn) time. O(min(n,m)) space.

def llcs5(a,b):
    '''Scores the Longest Common Subsequence between two strings'''
    return float(len(lcs5(a,b)))/float((min([len(a), len(b)])))

def lcs_lens(xs, ys):
    curr = list(itertools.repeat(0, 1 + len(ys)))
    for x in xs:
        prev = list(curr)
        for i, y in enumerate(ys):
            if x == y:
                curr[i + 1] = prev[i] + 1
            else:
                curr[i + 1] = max(curr[i], prev[i + 1])
    return curr

def lcs5(xs, ys):
    '''Calculates Longest Common Subsequnce'''
    nx, ny = len(xs), len(ys)
    if nx == 0:
        return ""
    elif nx == 1:
        return xs[0] if xs[0] in ys else ""
    else:
        i = nx // 2
        xb, xe = xs[:i], xs[i:]
        ll_b = lcs_lens(xb, ys)
        ll_e = lcs_lens(xe[::-1], ys[::-1])
        
        temp = [(ll_b[j] + ll_e[ny - j], j) for j in range(ny + 1)]
        _, k = max(temp)

        yb, ye = ys[:k], ys[k:]
        return lcs5(xb, yb) + lcs5(xe, ye)

In [9]:
# 3.5.1 Testing the complexity the lcs5
#print lcs5('human','chimpanzee')
print lcs5('ATCTGAT', 'TGCATA')

# Really long reads
longRead1 = "GGGGGCATGCCTTAACATGCAGTCGAACGGCAGCACGGGTGCTTGCACCTGGTGGCGAGTGGCGAACGGGTGAGTAATACATCGGAACATGTCCAGTAGTGGGGGATAGCCCGGCGAAAGCCGGATTAATACCGCATACGCTCTATGGAGGAAAGCGGGGGATCGAAAGACCTCGCGCTATAGGGGTGGCCGATGGCGGATTAGCTAGTTGGTGGGGTAAAGGCCTACCAAGGCGACGATCCGTAGCTGGTCTGAGAGGACGACCAGCCACACTGGGACTGAGACACGGCCCAGACTCCTACGGGAGGCAGCAGTGGGGAATTTTGGACAATGGGCGAAAGCCTGATCCAGCAATGCCGCGTGTGTGAAGAAGGCCTTCGGGTTGTAAAGCACTTTTGTCCGGAAAGAAATCCCTGGTCCTAATATGGCCGGGGGATGACGGTACCGGAAGAATAAGCACCGGCTAACTACGTGCCAGCAGCCGCGGTAATACGTAGGGTGCAAGCGTTAATCGGAATTACTGGGCGTAAAGCGTGCGCAGGCGGTGATGCAAGACCGATGTGAAATCCCCGGGCTCAACCTGGGAACTGCATTGGTGACTGCATCGCTAGAGTATGGCAGAGGGGGGTAGAATTCCACGTGTAGCAGTGAAATGCGTAGAGATGTGGAGGAATACCGATGGCGAAGGCAGCCCCCTGGGTCAATACTGACGCTCATGCACGAAAGCGTGGGGAGCAAACAGGATTAGATACCCTGGTAGTCCACGCCCTAAACGATGTCAACTGGTTGTCGGGTCTTCATTGACTTGGTAACGTAGCTAACGCGTGAAGTTGACCGCCTGGGGAGTACGGTCGCAAGATTAAAACTCAAAGGAATTGACGGGGACCCGCACAAGCGGTGGATGATGTGGATTAATTCGATGCAACGCGAAAAACCTTACCTACCCTTGACATGGATGGAATCCCGCTGAGAGGTGGGAGTGCCCGAAAGGGAGCCATCACACAGGTGCTGCATGGCTGTCGTCAGCTCGTGTCGTGAGATGTTGGGTTAAGTCCCGCAACGAGCGCAACCCTTGTCCCTAGTTGCTACGCAAGAGCACTCCAGGGAGACTGCCGGTGACAAACCGGAGGAAGGTGGGGATGACGTCAAGTCCTCATGGCCCTTATGGGTAGGGCTTCACACGTCATACAATGGTCGGAACAGAGGGTTGCCAAGCCGCGAGGTGGAGCCAATCCCAGAAAACCGATCGTAGTCCGGATCGCAGTCTGCAACTCGACTGCGTGAAGCTGGAATCGCTAGTAATCGCGGATCAGCATGCCGCGGTGAATACGTTCCCGGGTCTTGTACACACCGCCCGTCACACCATGGGAGTGGGTTTTGCCAGAAGTGGCTAGTCTAACCGCAAGGAGGACGGTCACCACGTCAGATCAGCGT"
longRead2 = "TAGACTGCAGTCGAACGCAGTCTTCGGACTGAGTGGCGCACGGGTGAGTAACGCGTAACTGACGTACCCGGAAGTCCAGAATAACTTCCCGAAAGGGAAGCTAATACCGGATGTGCAGCACTTCTTCTGGGAGTGTTGTAAAGATTTATCGCTTTCGGATCGGGTTGCGTTCCATCAGCTAGATGGTGAGGTAAAGGCTCACCATGGCAACGACGGATAGCCGACCTGAGAGGGTGGTCGGCCACAGGGGCACTGAGACACGGGCCCCACTCCTACGGGAGGCAGCAGTTAGGAATCTTCTGCAATGGACGAAAGTCTGACAGAGCGACGCCGCTTGAGGGAAGAAGGTCTTCGGATTGTAAACCTCTGAACTTGGGACGAAAGACCAGCTTGACTGGGGATGACGGTACCAAGGTAATAGCACCGGCTAACTCCGTGCCAGCAGCCGCGGTAATACGGAGGGTGCGAGCGTTACCCGGAATCACTGGGCGTAAAGGGCGTGTAGGCGGTTTTTCAAGTCTGGCTTTAAAGACCGGAGCTCAACTCCGGGAGTGGGCTGGATACTGGGAGACTTGACGGCTGGAGAGGCAACTGGAATTCCTGGTGTAGCGGTGGAATGCGTAGATACCAGGAGGAACACCGATGGCGAAGGCAGGTTGCTGGACAGCGAGTGACGCTGAGGCGCGAAAGTGTGGGGAGCGAACCGGATTAGATACCCGGGTAGTCCACACCCTAAACGATGCACGTTGGATTACCGTGGGATGCCATGGTGGTCGAAGCCAACGCGATAAACGTGCCGCCTGGGAAGTACGGCCGCAAGGTTGAAACTCAAAGGAATTGACGGGGGCCCGCACAAGCGGTGGAGCATGTGGTTTAATTCGAAGCAACGCGAAGAACCTTACCAGGTCTTGACATGCACGGAGCCTTCCTGAAAGGGAAGGGTGCCCTTCGGGGAACCGTGACACAGGTGCTGCATGGCTGTCGTCAGCTCGTGTCGTGAGATGTTGGGTTAAGTCCCGCAACGAGCGCAACCCTTGCTTTGTGTTACTAACGGTTTGGCCGAGGACTCACAAGGGACTGCCTGTGAAAGCAGGAGGAAGGCGGGGATGACGTCTAGTCAGCATGGTCCTTACGACCTGGGCTACACACGTGCTACAATGGTCGGTACAACGCGCAGCAACACCGCGAGGTGAAGCGAATCGCTGAAAGCCGGCCTCAGTTCAGATTGCAGTCTGCAACTCGACTGCATGAAGTTGGAATCGCTAGTAATCGCGGGTCAGCACACCGCGGTGAATACGTTCCCGGGCCTTGTACACACCGCCCGTCACACCATGGGAGTTGATTGCAGCTGAAGCCGCCGGGAGCCGCAAGGCAGGCGTCTAGGCTGTGGTTGATGACTGGGGTGAAGTCGTAACAA"
start = time.time()
print "LCS: ", "".join(lcs5(longRead1, longRead2))
end = time.time()
print "LCS Similarity: ", llcs5(longRead1, longRead2)
print "Run Time: ", end-start, " seconds"

TCTA
LCS:  AGACTGCAGTCGAACGCAGTCTTGACTGGTGGCGCACGGGTGAGTAATACATCGGAAGTCCAGATAACCCCGAAAGGGATAATACCGATACCTCTTGGGAGGGTAAAGACCTCGGATGGGTGGCCATCAGCTAGTGGTGGGTAAAGGCCACCAGGCGACGATCCGACCTGAGAGGCGGCCACAGGGACTGAGACACGGCCCACTCCTACGGGAGGCAGCAGTGGAATTTTGCAATGGCGAAAGCTGACAGCAGCCGCGGGGAAGAAGGCTTCGGTTGTAAACCTTTTGCGAAAGACCGCTTATGGGGATGACGGTACCAAGAATAGCACCGGCTAACTCGTGCCAGCAGCCGCGGTAATACGAGGGTGCAGCGTTACGGAATACTGGGCGTAAAGCGTGAGGCGGTTCAAGCCTTAAACCGGGCTCAACCTGGGCTGATTGGGACTGACGCTGAGAGGCAAGGAATTCCGTGTAGCGTGAATGCGTAGAAGGAGGAAACCGATGGCGAAGGCAGCTGGGCATGACGCTAGCCGAAAGGTGGGGAGCAACGGATTAGATACCCGGTAGTCCACCCCTAAACGATGCACTGGTTGTGGGTCCATGTGGTCGAGCAACGCGTAAGTGCCGCCTGGGAGTACGGCGCAAGTTAAACTCAAAGGAATTGACGGGGCCCGCACAAGCGGTGGAGATGTGGTTAATTCGAGCAACGCGAAAACCTTACCTCTTGACATGAGGACCCCTGAAGGGGGGTGCCCGGGGACCTACACAGGTGCTGCATGGCTGTCGTCAGCTCGTGTCGTGAGATGTTGGGTTAAGTCCCGCAACGAGCGCAACCCTTGTCTAGTTGCCGAGGACTCCAGGGACTGCCGTGAAAAGGAGGAAGGGGGGATGACGTCAGTCCATGGCCTTAGTGGGCTCACACGTCTACAATGGTCGGACAAGGGCCACCGCGAGGTGAGCAATCCGAAACCGTCGTTCGATGCAGTCTG

In [ ]:
# LCS Filter 1
def lcsFilter1(cluster, s): newCluster = [] counter = 1
    for clump in cluster:
        repCluster = []
        disClusters = []

        r_rep = clump[0]
        for q in clump:
            if llcs5(q,r_rep) >= s:
                repCluster.append(q)
            else:
                for disCluster in disClusters:
                    r_dis = disCluster[0]
                    if llcs5(q, r_dis) >= s:
                        disCluster.append(q)
                        break
                else:
                    disClusters.append([q])
        newCluster.append(["Cluster " + str(counter), repCluster, disClusters])
        counter = counter + 1
    return newCluster

In [12]:
# LCS Filter 2
def averageCluster(cluster):
    if len(cluster) < 2:
        return cluster
    else:
        curr_avg = "".join(lcs5(cluster[0], cluster[1]))
        
        for i in range(2, len(cluster)):
            print curr_avg
            curr_avg = "".join(lcs5(curr_avg, cluster[i]))
            
        return curr_avg
            
def lcsFilter2(cluster, s):
    newCluster = []
    counter = 1
    
    print averageCluster(cluster)

In [13]:
mini = 4; maxi = 4; s_threshold = 0.85
filename = 'f2014_hw4_reads.fa'

# 1. Reads .fa file
text = open(filename)
reads = getReads(text)

# 2. Creats an index
index = makeIndex(reads[0], [], mini, maxi+1)

# 3. Short Word filter
shortWordCluster = shortWordFilter(reads, index, mini, maxi)

In [14]:
# 4. Testing LCS Filter
testCluster = [shortWordCluster[7]]
lcsCluster = lcsFilter2(testCluster, 0.95)

[['ACTTCATGGCAGCCGACTCTCTGTAGCTCTATAGCCGTCCAGGCGAAGGTCAATACCTGTGACTGCTTCCTTCTAGCTCACATGAGTGTTTCGCATTCGG', 'GTACTTCATGGCAGCCGACTCTCTGTAGCTCTATAGCCGTCCAGGCGAAGGTCAATACCTGTGACTGCTTCCTTCTAGCTCACATGAGTGTTTCGCATTC', 'GGCAGCCGACTCTCTGTAGCTCTATAGCCGTCCAGGCGAAGGTCAATACCTGTGACTGCTTCCTTCTAGCTCACATGAGTGTTTCGCATTCGGTAAACTA', 'GTACGTACTTCATGGCAGCCGACTCTCTGTAGCTCTATAGCCGTCCAGGCGAAGGTCAATACCTGTGACTGCTTCCTTCTAGCTCACATGAGTGTTTCGC', 'ATGGCAGCCGACTCTCTGTAGCTCTATAGCCGTCCAGGCGAAGGTCAATACCTGTGACTGCTTCCTTCTAGCTCACATGAGTGTTTCGCATTCGGTAAAC', 'GAGTACGTACTTCATGGCAGCCGACTCTCTGTAGCTCTATAGCCGTCCAGGCGAAGGTCAATACCTGTGACTGCTTCCTTCTAGCTCACATGAGTGTTTC', 'CATGGCAGCCGACTCTCTGTAGCTCTATAGCCGTCCAGGCGAAGGTCAATACCTGTGACTGCTTCCTTCTAGCTCACATGAGTGTTTCGCATTCGGTAAA', 'ACGTACTTCATGGCAGCCGACTCTCTGTAGCTCTATAGCCGTCCAGGCGAAGGTCAATACCTGTGACTGCTTCCTTCTAGCTCACATGAGTGTTTCGCAT', 'TACTTCATGGCAGCCGACTCTCTGTAGCTCTATAGCCGTCCAGGCGAAGGTCAATACCTGTGACTGCTTCCTTCTAGCTCACATGAGTGTTTCGCATTCG']]


In [15]:
# Example of the first cluster from short word filter
swCluster = shortWordCluster[0]
swCluster

['CTGGTTGGTGTAGATAAGCCAAGATCGAAGCTCGAGCGAGAGCTAGTGTTGGTGCCCGGAGCCATGGCGGGTATGTCACGCCAATCAGCGTACAAGGAGG',
 'AGATAAGCCAAGATCGAAGCTCGAGCGAGAGCTAGTGTTGGTGCCCGGAGCCATGGCGGGTATGTCACGCCAATCAGCGTACAAGGAGGAAATCGCAATC',
 'ATCGCCAAATCTGGTTGGTGTAGATAAGCCAAGATCGAAGCTCGAGCGAGAGCTAGTGTTGGTGCCCGGAGCCATGGCGGGTATGTCACGCCAATCAGCG',
 'ACGTATCGCCAAATCTGGTTGGTGTAGATAAGCCAAGATCGAAGCTCGAGCGAGAGCTAGTGTTGGTGCCCGGAGCCATGGCGGGTATGTCACGCCAATC',
 'GTATCGCCAAATCTGGTTGGTGTAGATAAGCCAAGATCGAAGCTCGAGCGAGAGCTAGTGTTGGTGCCCGGAGCCATGGCGGGTATGTCACGCCAATCAG',
 'GTAGATAAGCCAAGATCGAAGCTCGAGCGAGAGCTAGTGTTGGTGCCCGGAGCCATGGCGGGTATGTCACGCCAATCAGCGTACAAGGAGGAAATCGCAA',
 'AAATCTGGTTGGTGTAGATAAGCCAAGATCGAAGCTCGAGCGAGAGCTAGTGTTGGTGCCCGGAGCCATGGCGGGTATGTCACGCCAATCAGCGTACAAG',
 'CAAATCTGGTTGGTGTAGATAAGCCAAGATCGAAGCTCGAGCGAGAGCTAGTGTTGGTGCCCGGAGCCATGGCGGGTATGTCACGCCAATCAGCGTACAA',
 'GCCAAATCTGGTTGGTGTAGATAAGCCAAGATCGAAGCTCGAGCGAGAGCTAGTGTTGGTGCCCGGAGCCATGGCGGGTATGTCACGCCAATCAGCGTAC',
 'AATCTGGTTGGTGTAGATAAGCCAAGATCGAAGCTCGAGCGAGAGCTAGTGTT

In [18]:
#Example of the first cluster from lcs filter
lcsCluster = lcsFilter2(swCluster, 0.95)
lcsCluster

AGATAAGCCAAGATCGAAGCTCGAGCGAGAGCTAGTGTTGGTGCCCGGAGCCATGGCGGGTATGTCACGCCAATCAGCGTACAAGGAGG
AGATAAGCCAAGATCGAAGCTCGAGCGAGAGCTAGTGTTGGTGCCCGGAGCCATGGCGGGTATGTCACGCCAATCAGCG
AGATAAGCCAAGATCGAAGCTCGAGCGAGAGCTAGTGTTGGTGCCCGGAGCCATGGCGGGTATGTCACGCCAATC
AGATAAGCCAAGATCGAAGCTCGAGCGAGAGCTAGTGTTGGTGCCCGGAGCCATGGCGGGTATGTCACGCCAATC
AGATAAGCCAAGATCGAAGCTCGAGCGAGAGCTAGTGTTGGTGCCCGGAGCCATGGCGGGTATGTCACGCCAATC
AGATAAGCCAAGATCGAAGCTCGAGCGAGAGCTAGTGTTGGTGCCCGGAGCCATGGCGGGTATGTCACGCCAATC
AGATAAGCCAAGATCGAAGCTCGAGCGAGAGCTAGTGTTGGTGCCCGGAGCCATGGCGGGTATGTCACGCCAATC
AGATAAGCCAAGATCGAAGCTCGAGCGAGAGCTAGTGTTGGTGCCCGGAGCCATGGCGGGTATGTCACGCCAATC
AGATAAGCCAAGATCGAAGCTCGAGCGAGAGCTAGTGTTGGTGCCCGGAGCCATGGCGGGTATGTCACGCCAATC
AGATAAGCCAAGATCGAAGCTCGAGCGAGAGCTAGTGTTGGTGCCCGGAGCCATGGCGGGTATGTCACGCCAATC
AGATAAGCCAAGATCGAAGCTCGAGCGAGAGCTAGTGTTGGTGCCCGGAGCCATGGCGGGTATGTCACGCCAATC
AGATAAGCCAAGATCGAAGCTCGAGCGAGAGCTAGTGTTGGTGCCCGGAGCCATGGCGGGTATGTCACGCCAATC
AGATAAGCCAAGATCGAAGCTCGAGCGAGAGCTAGTGTTGGTGCCCGGAGCCATGGCGGGTATGTCACGC

In [16]:
# Cluster Comparison
filename1 = 'phanicluster.txt'
filename2 = 'cdhitcluster.txt'

# 1. Reads .fa file
text1 = open(filename1)
reads1 = text1.readlines()
text2 = open(filename2)
reads2 = text2.readlines()

readIDsPhani = set()
readIDsCDHit = set()
for reads in reads1:
    readIDsPhani.add(reads.split("|")[1])
for reads in reads2:
    readIDsCDHit.add(reads.split("|")[1])
print len(readIDsPhani.intersection(readIDsCDHit))

31
